# 快速上手示例：构建一个简单的问答系统

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os
load_dotenv()

# 初始化模型
# 可以使用OpenAI的API
# llm = ChatOpenAI(model="gpt-4", api_key="your-openai-api-key")
# 或者可以用其他的API，比如阿里云的
api_key = os.getenv("api_key")
base_url = os.getenv("base_url")
model = os.getenv("model")
llm = ChatOpenAI(api_key=api_key, base_url=base_url,model=model,  streaming=True)

###################调用方法一########################
result = llm.stream("2025年的技术趋势是什么？")
for chunk in result:
    print(chunk.content, end='', flush=True)
###################################################


###################调用方法二##########################
# 定义提示模板
prompt = PromptTemplate(
    input_variables=["question"],
    template="请简洁回答以下问题：{question}"
)

# 创建链
chain = LLMChain(llm=llm, prompt=prompt)

# 运行链
question = "2025年的技术趋势是什么？"
response = chain.run(question)
print(response)
######################################################

<generator object BaseChatModel.stream at 0x000001AE27B42260>
2025年的技术趋势将聚焦以下关键领域：  
1. **人工智能深化**：生成式AI、多模态模型进一步成熟，推动医疗、制造、教育等垂直领域应用。  
2. **量子计算突破**：实用化进展加速，可能在药物研发、优化问题解决中实现初步商业化。  
3. **绿色技术主导**：可再生能源、储能技术、碳捕捉及氢能技术加速落地，助力碳中和目标。  
4. **生物技术革命**：基因编辑（如CRISPR）推动个性化医疗，合成生物学或重塑工业生产。  
5. **扩展现实（XR）普及**：AR/VR硬件轻量化，应用场景扩展至远程协作、教育及元宇宙生态。  
6. **边缘计算与6G**：低延迟边缘计算结合5G/6G网络，支持物联网与实时数据处理需求。  
7. **网络安全强化**：零信任架构与AI驱动的威胁检测成为企业刚需，应对复杂攻击。  
8. **脑机接口探索**：神经科技在医疗领域（如瘫痪治疗）展开临床应用，长期潜力显著。  
9. **区块链与Web3**：企业级应用聚焦供应链透明化与去中心化金融（DeFi）创新。  

这些趋势将共同推动社会效率提升与可持续发展，同时带来伦理与安全挑战。


# 进阶功能：RAG 系统

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
load_dotenv()
# 加载文档
loader = TextLoader("example_doc.txt")
documents = loader.load()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")


embeddings = DashScopeEmbeddings(
    model="text-embedding-v3",
    dashscope_api_key=api_key,
    # base_url=base_url
)



vector_store = FAISS.from_documents(documents, embeddings)

# 初始化检索链
qa_chain = RetrievalQA.from_chain_type(
     
    llm=ChatOpenAI(api_key=api_key, base_url=base_url,model="qwq-32b",streaming=True),
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# 查询
response = qa_chain.run("文档中提到的重要内容是什么？")
print(response)


根据您提供的文档内容，以下是提到的重要内容总结：

---

### **核心模块与功能**
1. **模型 I/O**  
   - 与 LLM 交互，支持 OpenAI、阿里云（如 `qwq-32b`）、Anthropic 等模型。
   - 示例：通过 `ChatOpenAI` 初始化模型，调用 `llm.invoke()` 或 `chain.run()` 获取响应。

2. **内存（Memory）**  
   - 保存对话历史，例如 `ConversationBufferMemory` 可记住用户之前的输入。
   - 示例：在 `ConversationChain` 中使用内存，实现多轮对话的上下文感知。

3. **链（Chains）**  
   - 组合提示、模型和逻辑，例如 `LLMChain` 处理单一流程，`SequentialChain` 连接多个链。
   - 示例：通过 `PromptTemplate` 定义问题模板，结合模型生成回答。

4. **检索（Retrieval）**  
   - 通过向量数据库（如 FAISS 或 Qdrant）实现检索增强生成（RAG）。
   - 示例：加载外部文档，使用 `OpenAIEmbeddings` 创建向量存储，通过 `RetrievalQA` 回答基于文档的问题。

5. **代理（Agents）**  
   - 让 LLM 动态调用工具（如维基百科搜索、计算器等）解决问题。
   - 示例：使用 `initialize_agent` 结合 `WikipediaAPIWrapper` 获取实时信息。

6. **LangGraph**  
   - 构建复杂工作流或多代理系统，支持状态管理和节点间逻辑。
   - 示例：定义状态和节点函数，通过 `StateGraph` 编排流程。

---

### **关键代码示例**
1. **基础问答系统**  
   ```python
   # 使用阿里云模型
   llm = ChatOpenAI(
       api_key=os.getenv("api_key"),
       base_url=os.getenv("base_url"),
       model=os.getenv("model"),
       stre